### Word Group In Youtube Firebase

In [4]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [5]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [6]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [7]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [8]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [9]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Quran With Flashcards/Api Key/quran-with-flashcards-firebase-adminsdk-t8xhw-202e2c7561.json")  # Quran with Flashcards
app = firebase_admin.initialize_app(cred)

In [10]:
fire_db = firestore.client()

#### Read Data

In [11]:
df_28 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ne o beni ne de ben onu bir daha,3870,3873,bundan böyle ne o beni ne de ben onu bir daha ...,FUS0hsPCtMs,https://www.youtube.com/watch?v=FUS0hsPCtMs&t=...
1,bu kadar mı hayır bu kadar değil,761,764,ölüm bir terhistir dedik terhis olan mutlu olu...,vOWbfQ0tIyU,https://www.youtube.com/watch?v=vOWbfQ0tIyU&t=...
2,var değil mi mi ben çok çok yok,84,86,ülkelerde yoğunluk var değil mi mi ben çok çok...,cUfp2lydmho,https://www.youtube.com/watch?v=cUfp2lydmho&t=84s
3,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
4,çok bir şey değil ama ben bunu,1663,1665,çok bir şey değil ama ben bunu almanı istiyorum,EtWqWE8gdF0,https://www.youtube.com/watch?v=EtWqWE8gdF0&t=...
...,...,...,...,...,...,...
762,da var değil mi ama,4685,4688,şeyin arkadaşlar da var değil mi ama inan,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
763,mı seni o kadar çok,8117,8119,ondan mı seni o kadar çok aradı o ya,orEhowBkuKI,https://www.youtube.com/watch?v=orEhowBkuKI&t=...
764,o kadar çok şey var,605,607,yapacağımız o kadar çok şey var ki,5ONPIsJtBOg,https://www.youtube.com/watch?v=5ONPIsJtBOg&t=...
765,ama değil mi sen ne,3395,3397,kızını oğluma yamamaya çalışırken iyidi ama de...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...


In [12]:
df_200 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
1,bilmiyorum sen ne demek istiyorsun ben de onu ...,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...,ZdeFof81xKs,https://www.youtube.com/watch?v=ZdeFof81xKs&t=...
2,var da bu bir sorun olur mu benim için ben bun...,7920,7926,var da bu bir sorun olur mu benim için ben bun...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
4,gerçekten çok teşekkür ederim ya bu çok büyük ...,2257,2262,gerçekten çok teşekkür ederim ya bu çok büyük ...,a8Dov7B7alQ,https://www.youtube.com/watch?v=a8Dov7B7alQ&t=...
...,...,...,...,...,...,...
5487,en kötü ve en iyi,606,608,en kötü ve en iyi senaryolarına göre ölçeklend...,Whh9HzTIOyE,https://www.youtube.com/watch?v=Whh9HzTIOyE&t=...
5488,en iyi ve en yeni,1145,1148,en iyi ve en yeni planımızı yapıyoruz,ORAtNZJoCBo,https://www.youtube.com/watch?v=ORAtNZJoCBo&t=...
5489,şu al bunu ve şu,1385,1386,şu al bunu ve şu kıyafetini almak o yüzden şu ...,7fsccwLqxL8,https://www.youtube.com/watch?v=7fsccwLqxL8&t=...
5490,ve şu en son ki,697,699,ve şu en son ki batmış olduğumuz trabzandan so...,Bt5YkKBrL4k,https://www.youtube.com/watch?v=Bt5YkKBrL4k&t=...


In [13]:
df_1000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,https://www.youtube.com/watch?v=eqkDognLtTc&t=...
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
...,...,...,...,...,...,...
15359,boş ver be baba i,7310,7312,boş ver be baba i şin kaynağı sende değil mi,EFEFEcYl9es,https://www.youtube.com/watch?v=EFEFEcYl9es&t=...
15360,el ile el ele göz,1467,1469,kızımız yedi kat el ile el ele göz göze gelmiş,0cCZkBCbZco,https://www.youtube.com/watch?v=0cCZkBCbZco&t=...
15361,beş bin su on bin,4632,4635,elektrik yirmi beş bin su on bin,UDlsIakb9Kg,https://www.youtube.com/watch?v=UDlsIakb9Kg&t=...
15362,e sağ ol be baba,3280,3282,e sağ ol be baba,vMxEQFO8teQ,https://www.youtube.com/watch?v=vMxEQFO8teQ&t=...


#### Sending Data To Firestore

##### Absolute Beginner

In [11]:
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ne o beni ne de ben onu bir daha,3870,3873,bundan böyle ne o beni ne de ben onu bir daha ...,FUS0hsPCtMs,https://www.youtube.com/watch?v=FUS0hsPCtMs&t=...
1,bu kadar mı hayır bu kadar değil,761,764,ölüm bir terhistir dedik terhis olan mutlu olu...,vOWbfQ0tIyU,https://www.youtube.com/watch?v=vOWbfQ0tIyU&t=...
2,var değil mi mi ben çok çok yok,84,86,ülkelerde yoğunluk var değil mi mi ben çok çok...,cUfp2lydmho,https://www.youtube.com/watch?v=cUfp2lydmho&t=84s
3,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
4,çok bir şey değil ama ben bunu,1663,1665,çok bir şey değil ama ben bunu almanı istiyorum,EtWqWE8gdF0,https://www.youtube.com/watch?v=EtWqWE8gdF0&t=...
...,...,...,...,...,...,...
762,da var değil mi ama,4685,4688,şeyin arkadaşlar da var değil mi ama inan,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
763,mı seni o kadar çok,8117,8119,ondan mı seni o kadar çok aradı o ya,orEhowBkuKI,https://www.youtube.com/watch?v=orEhowBkuKI&t=...
764,o kadar çok şey var,605,607,yapacağımız o kadar çok şey var ki,5ONPIsJtBOg,https://www.youtube.com/watch?v=5ONPIsJtBOg&t=...
765,ama değil mi sen ne,3395,3397,kızını oğluma yamamaya çalışırken iyidi ama de...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...


In [12]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [21]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description = {"description":"This description", "level":0, "levelName":"Absolute Beginner"}

In [22]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)

#beginner_data_list   

In [ ]:
data_beginner = {"datas":{"0":beginner_description,"videos":beginner_data_list}}
#data_beginner

In [ ]:
#data_all = {"datas":{"0":beginner_description,"videos":beginner_data_list},{"1":A1_description,"videos":A1_data_list},{"2":A2_description,"videos":A2_data_list},\
#{"3":B1_description,"videos":B1_data_list},{"4":B2_description,"videos":B2_data_list},{"5":C1_description,"videos":C1_data_list},{"6":C2_description,"videos":C2_data_list}}
##data_all  # will test

In [26]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

update_time {
  seconds: 1669038749
  nanos: 661916000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000